**Текущие задачи**:
- Описать `requirements` или сделать Docker-образ
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.

# Парсер

Реализованы парсеры данных и собраны тексты и метаинформация статей следующих ресурсов:
- Хабрахабр https://habr.com/
- Типичный программист https://tproger.ru/

Исходные данные собраны посредством `wget` и уже предобработаны. Однако со временем на ресурсах публикуются новые статьи, а старые статьи продолжают набирать просмотры. Сбор и обновление соответствующей информации помогают учесть появление новых технологий.

Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, необходимо их опрашивать поочередно. Все необходимые методы содержатся внутри класса `Parser`, а оркестровку парсеров проводит объект класса `ParserComposer`.

Системные требования:
- Каталог с датасетами
- Дравйвер `/usr/bin/chromedriver`

In [1]:
# standard library
import random
import datetime
import pickle
import os
import re
import uuid

# conections
import requests
from requests.exceptions import ConnectionError

# tor network for private parsing with privoxy and stem
# https://gist.github.com/KhepryQuixote/46cf4f3b999d7f658853
from stem import Signal
from stem.control import Controller

# for tor network IP changing
with Controller.from_port(port = 9051) as controller:
    controller.authenticate('07011951')
    controller.signal(Signal.NEWNYM)

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.73.11 (KHTML, like Gecko) Version/7.0.1 Safari/537.73.11'
}

# parsers libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
        {
            'base_url': 'https://tproger.ru/',
            'first_page_url': 'https://tproger.ru/page/1/',
            'find_all_args': ('a', {'class':'article-link'})
        },
    'habr':
        {
            'base_url': 'https://habr.com/',
            'first_page_url': 'https://habr.com/ru/page1/',
            'find_all_args': ('a', {'class':'post__title_link'})
        }
}

proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}

# ссылки на новые статьи, которые не были обработаны ранее
with open('not_processed_urls.pickle', 'rb') as f:
    nurls = pickle.load(f)

DATASETS_PATH = '/home/leo/DATASETS'

for source_name in sources:
    sources[source_name]['df'] = pd.read_csv(f'{DATASETS_PATH}/{source_name}.csv',
                                             index_col=0,
                                             parse_dates=['post_time', 'parse_time'])

In [2]:
class Parser:
    def __init__(self, source_name):
        self.source_name = source_name
        self.source = sources[source_name]
        self.df = pd.read_csv(f'DATASETS_PATH/{source_name}.csv',
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])
    
    def delta_dates_days(self):
        today_date = datetime.datetime.now()
        last_parse_date = self.df.post_time.max()
        today_date = today_date.replace(tzinfo=last_parse_date.tz)
        delta_dates = today_date - last_parse_date
        return delta_dates.days
    
    def check_new_articles(self):
        '''Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.'''
        all_new_urls = nurls[self.source_name]
        page_count = 1
        source = self.source

        attempts_counts = 0
        while True:
            try:
                page_url = source['first_page_url'].replace('1', str(page_count))
                page = requests.get(page_url,
                                    headers=headers,
                                    proxies=proxies,
                                    stream=False)
                soup = BeautifulSoup(page.text, 'html.parser')
                urls = {url['href'] for url in soup.find_all(*source['find_all_args'])}
                if not urls:
                    # the thread processes the case when captcha and no urls
                    print("Changing proxy because of no urls or CAPTCHA...")
                    attempts_counts += 1
                    if attempts_counts >= 2:
                        print(f"Data parsing of {self.source_name} is stopped at {page_url}.")
                        break
                    proxies = None
                    continue
                old_urls = set(source['df'].index.to_list())
                new_urls = urls - old_urls
                all_new_urls = all_new_urls | new_urls
                nurls[self.source_name] = all_new_urls
                if len(new_urls) > 0:
                    print(f"{len(new_urls)} urls are collected from {page_url}")
                if (len(new_urls) == 0) or (len(urls) == 0):    
                    print(f"{len(all_new_urls)} new urls are saved for {source['base_url']}")
                    break
                else:
                    page_count += 1
            except ConnectionError:
                print("Problems with connection...")
                continue
        with open('not_processed_urls.pickle', 'wb') as f:
            pickle.dump(nurls, f)


    def save_new_data(self, url, data, full_text):
        '''Сохраняет новые данные в датафрейм, а текст -- в отдельный файл'''
        data['filename'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, url))
        new_row = pd.DataFrame(data, index=[url])
        if url not in self.df:
            self.df = self.df.append(new_row)
        else:
            self.df.iloc[url] = new_row
        
        filepath = f"{DATASETS_PATH}/{self.source_name}/{data['filename']}"
        if not os.path.exists(filepath):
            with open(filepath, 'w') as f:
                full_text = re.sub('\n+', '\n\n', full_text).strip()
                f.write(full_text)
                print(f"File {self.source_name}/{data['filename']} is saved.")
        else:
            print(f"File {self.source_name}/{data['filename']} is already exist.")
            
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class HabrParser(Parser):
    '''Парсер для сайта habr.com'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        page = requests.get(url, headers=headers, proxies=proxies, stream=False)
        soup = BeautifulSoup(page.text, 'html.parser')
        data['title'] = soup.select_one('h1').text.strip()
        date_time = soup.select_one('.post__time').get('data-time_published')
        data['post_time'] = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%MZ')
        data['views_num'] = soup.select_one('.post-stats__views-count').text.strip()
        data['likes_num'] = int(soup.select_one('.js-post-vote').text.strip().replace('–', '-'))
        data['favs_num'] = int(soup.select_one('.bookmark__counter').text)
        try:
            data['comments_num'] = int(soup.select_one('.post-stats__comments-count').text)
        except AttributeError:
            data['comments_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        full_text = soup.select_one('.post__body_full').text
        self.save_new_data(url, data, full_text)


class TprogerParser(Parser):
    '''Парсер для сайта tproger.ru'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        try:
            driver = webdriver.Chrome('/usr/bin/chromedriver')
            driver.get(url)
            page = driver.page_source
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.post-views-count'))
            WebDriverWait(driver, timeout=10).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        finally:
            driver.quit()

        soup = BeautifulSoup(page, 'html.parser')
        data['title'] = soup.h1.text.strip()
        data['post_time'] = datetime.datetime.fromisoformat(soup.select_one('time')['content'])
        paragraphs = soup.find_all('div', {'class':'entry-content'})[0].find_all('p')
        try:
            data['short_text'] = paragraphs[0].text
        except IndexError:
            data['short_text'] = ''
        try:
            data['views_num'] = soup.select_one('.post-views-count').text
        except AttributeError:
            data['views_num'] = 0
        full_text = "\n\n".join(p.text for p in paragraphs[:-1])
        self.save_new_data(url, data, full_text)    
        
        
class ParserComposer:
    def __init__(self):
        pass
    
    def scheduler(self):
        '''
        Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам.
        Порядок проверки:
        1) загружаем те статьи, что не были обработаны по каким-то причинам ранее,
        2) проверяем, не вышли ли новые статьи
        3) обновляем число просмотров статей, вышедших в последний месяц (post_time)
        4) обновляем число просмотров статей, проверявшихся > месяца назад (parse_time), не относящихся к п. 2
        '''
        pass
    
    def pipeline(self):
        '''Проверяем, вышли ли новые статьи. Планируем загрузки. Обновляем датасеты.'''
        pass

In [3]:
habr_parser = HabrParser('habr')
#habr_parser.check_new_articles()

In [4]:
tproger_parser = TprogerParser('tproger')
#tproger_parser.check_new_articles()

In [ ]:
def load_new_articles(nurls):
    pairs = [(key, item) for key in nurls for item in nurls[key]]
    random.shuffle(pairs)
    for pair in tqdm(pairs):
        eval(f"{pair[0]}_parser.update({'pair[1]'})")

load_new_articles(nurls)

  0%|          | 0/1021 [00:00<?, ?it/s]

File habr/a7117495-0873-5040-a46a-0c8c3ce35d6b is saved.
File habr/9663f7f9-de9d-55fb-afa1-d798a89be278 is saved.
File habr/f38f9c1c-16e0-52a1-8ddf-18d6700fc034 is saved.
File tproger/a68a1558-f1f4-5a8e-bbbf-652890ae6f30 is already exist.
File habr/3394b21b-3734-5797-bc49-a3fce8b21951 is saved.
File habr/96b6b39c-6365-5f5b-aa63-294d83111d6a is saved.
File habr/983e1ebd-dfed-5db8-b351-c248f5c7865d is saved.
File habr/0e2f45f6-3493-5ef4-866f-acf5b11234f0 is saved.
File habr/0cc7caee-7605-5c1f-a043-b1b62c282c0b is saved.
File habr/9558ccc3-7de1-59c1-8708-48c244cfff68 is saved.
File habr/06a1f86a-d59f-5344-bdc0-8132857754ee is saved.
File habr/d31f6bcb-769a-5f8a-91a3-f3e39935011a is saved.
File habr/86530acd-1021-525d-8dd8-cffb61af3f43 is saved.
File habr/37508169-10b6-54c7-8c98-f144ef8b16f7 is saved.
File habr/21c754bd-5d48-53f5-adb2-cd4588943d9b is saved.
File habr/c70fd72f-4299-53ff-8a23-a0432df1afce is saved.
File habr/0587239a-0a2d-5b03-86fd-76814c270088 is saved.
File habr/6c9bef6e-4

In [ ]:
with open('not_processed_urls.pickle', 'wb') as f:
    pickle.dump(nurls, f)